In [1]:
import os
import ccxt
from configparser import ConfigParser
from binance.client import Client
from utils.functions import *

In [2]:
CREDENTIALS_PATH = os.path.join(os.path.expanduser('~'), ".binance/credentials")
config = ConfigParser()
config.read(CREDENTIALS_PATH)
api_key = config.get('trade', 'API_KEY')
api_secret  = config.get('trade', 'SECRET_KEY')

In [3]:
# Initialize Variables
CANDLE_DURATION_IN_MIN = 30

RSI_PERIOD = 14
RSI_OVERBOUGHT = 55
RSI_OVERSOLD = 45

CCXT_TICKER_NAME = 'LUNA/USDT'
TRADING_TICKER_NAME = 'LUNAUSDT'

INVESTMENT_AMOUNT_DOLLARS = 10
HOLDING_QUANTITY = 0

exchange = ccxt.binance()
binance_client = Client(api_key=api_key, api_secret=api_secret)
DECIMALS_QUANTITY = check_decimals(TRADING_TICKER_NAME, binance_client)

## Observations:
* It is recommend to initialize the bot close to the moment that a candle is closed

## Getting the data

In [4]:
df = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)
df

,at,open,high,low,close,vol,Date,symbol
0,1650981600000,95.20,95.33,93.71,94.30,185941.79,2022-04-26 14:00:00,LUNA/USDT
1,1650983400000,94.30,94.47,91.98,92.17,238420.42,2022-04-26 14:30:00,LUNA/USDT
2,1650985200000,92.16,92.70,91.53,91.83,221436.91,2022-04-26 15:00:00,LUNA/USDT
3,1650987000000,91.82,91.94,90.09,90.24,180142.31,2022-04-26 15:30:00,LUNA/USDT
4,1650988800000,90.25,91.40,89.58,89.97,323358.15,2022-04-26 16:00:00,LUNA/USDT
...,...,...,...,...,...,...,...,...
94,1651150800000,91.31,91.36,90.57,91.08,90868.43,2022-04-28 13:00:00,LUNA/USDT
95,1651152600000,91.07,91.31,89.29,89.37,162795.90,2022-04-28 13:30:00,LUNA/USDT
96,1651154400000,89.37,89.37,88.59,88.67,152543.21,2022-04-28 14:00:00,LUNA/USDT
97,1651156200000,88.64,88.85,87.95,88.75,130855.32,2022-04-28 14:30:00,LUNA/USDT


## Getting the trade recommendation

In [5]:
TRAD_REC_TYPE = get_trade_recommendation(df, RSI_OVERSOLD, RSI_OVERBOUGHT)

MACD Result: WAIT Final Result: WAIT


## Executing the trade

In [6]:
execute_trade(binance_client, TRAD_REC_TYPE, TRADING_TICKER_NAME, INVESTMENT_AMOUNT_DOLLARS, DECIMALS_QUANTITY)


ALERT!!! UNABLE TO COMPLETE ORDER


False

arrumar questão do currently_holding, criar stop loss, após atingir a alta 

In [7]:
currently_holding = False
while 1:
    # STEP 1: FETCH THE DATA
    ticker_data = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)
    if ticker_data is not None:
        # STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
        trade_rec_type = get_trade_recommendation(df, RSI_OVERSOLD, RSI_OVERBOUGHT)
        print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")} TRADING RECOMMENDATION: {trade_rec_type}')

        # STEP 3: EXECUTE THE TRADE
        if (trade_rec_type == 'BUY' and not currently_holding) or \
            (trade_rec_type == 'SELL' and currently_holding):
            print(f'Placing {trade_rec_type} order')
            trade_successful = execute_trade(binance_client, TRAD_REC_TYPE, TRADING_TICKER_NAME, INVESTMENT_AMOUNT_DOLLARS, DECIMALS_QUANTITY)
            currently_holding = not currently_holding if trade_successful else currently_holding

        time.sleep(CANDLE_DURATION_IN_MIN*60)
    else:
        print(f'Unable to fetch ticker data - {CCXT_TICKER_NAME}. Retrying!!')
        time.sleep(5)


MACD Result: WAIT Final Result: WAIT
28/04/2022 12:31:48 TRADING RECOMMENDATION: WAIT
